In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movie_df = pd.read_csv("/content/movies.csv")
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df = pd.read_csv("/content/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
ratings_df['userId'].value_counts()

,count
userId,
668,5678
575,2837
458,2086
232,1421
310,1287
...,...
58,20
51,20
288,20


In [ ]:
ratings_df = ratings_df.drop(['timestamp'], axis=1)
average_ratings_df = ratings_df.groupby(['movieId']).mean().reset_index()
average_ratings_df.head()


,movieId,userId,rating
0,1,335.663793,3.907328
1,2,338.173913,3.353261
2,3,299.982759,3.189655
3,4,342.727273,2.818182
4,5,284.822581,3.250000


In [ ]:
avg_ratings_df = ratings_df.groupby('movieId').agg({'rating': 'mean', 'userId': 'count'}).reset_index()
avg_ratings_df.head()

,movieId,rating,userId
0,1,3.907328,232
1,2,3.353261,92
2,3,3.189655,58
3,4,2.818182,11
4,5,3.250000,62


In [ ]:
Final_df = pd.merge(movie_df, avg_ratings_df, how='left', on='movieId')
Final_df.head()

,movieId,title,genres,rating,userId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.907328,232.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.353261,92.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.189655,58.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.818182,11.0
4,5,Father of the Bride Part II (1995),Comedy,3.250000,62.0


In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
def recommend_movies(genre, threshold, n):
  genre_df = Final_df[Final_df['genres'].str.contains(genre)]
  popular_df = genre_df[genre_df['userId'] >= threshold]
  popular_df = popular_df.sort_values(by='rating', ascending=False)
  return popular_df.head(n)
genre = input("Enter a genre: ")
threshold = int(input("Enter a minimum reviews threshold: "))
n = int(input("Enter the number of recommendations: "))
recommendations = recommend_movies(genre, threshold, n)
print("Here are {} recommendations for movies in the genre {}:".format(n, genre))
for movie in recommendations['title']:
  print(movie)

Enter a genre: Fantasy
Enter a minimum reviews threshold: 3
Enter the number of recommendations: 6
Here are 6 recommendations for movies in the genre Fantasy:
Nausicaä of the Valley of the Wind (Kaze no tani no Naushika) (1984)
About Time (2013)
Princess Mononoke (Mononoke-hime) (1997)
Akira Kurosawa's Dreams (Dreams) (1990)
Thief of Bagdad, The (1940)
My Neighbor Totoro (Tonari no Totoro) (1988)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tf_vectorizer = TfidfVectorizer(stop_words='english')
tf_vectorizer.fit(Final_df['genres'])
tf_matrix = tf_vectorizer.transform(Final_df['genres'])
cosine_sim = cosine_similarity(tf_matrix)
def recommend_movies(title, n):
  idx = movie_df[movie_df['title'] == title].index[0]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:n+1]
  movies = []
  for i in sim_scores:
    movies.append(movie_df.iloc[i[0]]['title'])
  return movies
title = input("Enter a movie title: ")
n = int(input("Enter the number of recommendations: "))
recommendations = recommend_movies(title, n)
print("Here are {} recommendations for movies:".format(n))
for movie in recommendations:
  print(movie)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter a movie title: Toy Story (1995)
Enter the number of recommendations: 2
Here are 2 recommendations for movies:
Antz (1998)
Toy Story 2 (1999)


In [ ]:
def user_input():
  genre = input("Enter a genre: ")
  threshold = int(input("Enter a minimum reviews threshold: "))
  n = int(input("Enter the number of recommendations: "))
  return genre, threshold, n

def recommend_movies(genre, threshold, n):
  genre_df = Final_df[Final_df['genres'].str.contains(genre)]
  popular_df = genre_df[genre_df['userId'] >= threshold]
  popular_df = popular_df.sort_values(by='rating', ascending=False)
  return popular_df.head(n)

def main():
  while True:
    genre, threshold, n = user_input()
    recommendations = recommend_movies(genre, threshold, n)
    if len(recommendations) == 0:
      print("No recommendations found. Please try again.")
    else:
      print("Here are {} recommendations for movies in the genre {}:".format(n, genre))
      for movie in recommendations['title']:
        print(movie)
      break

if __name__ == "__main__":
  main()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter a genre: Comedy|Romance
Enter a minimum reviews threshold: 5
Enter the number of recommendations: 20
Here are 20 recommendations for movies in the genre Comedy|Romance:
Louis C.K.: Shameless (2007)
Splendor in the Grass (1961)
Louis C.K.: Chewed Up (2008)
Ricky Gervais Live: Animals (2003)
Blackadder's Christmas Carol (1988)
Louis C.K.: Live at The Comedy Store (2015)
Creature Comforts (1989)
Louis C.K.: Hilarious (2010)
Dr. Horrible's Sing-Along Blog (2008)
About Time (2013)
Kind Hearts and Coronets (1949)
Fanboys (2009)
It's a Mad, Mad, Mad, Mad World (1963)
How Green Was My Valley (1941)
Intouchables (2011)
Shall We Dance (1937)
Top Hat (1935)
Lives of Others, The (Das leben der Anderen) (2006)
Notorious (1946)
Monty Python and the Holy Grail (1975)
